# Indian Food Classification and Recognition Using CNN


In [ ]:
import os
import random
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid

In [ ]:
transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

# Train and Test Setting

In [ ]:
dataset=datasets.ImageFolder(root=("/kaggle/input/indian-food-images-dataset/Indian Food Images/Indian Food Images"),transform=transform)

In [ ]:
class_names=dataset.classes
print(class_names)
print(len(class_names))

In [ ]:
torch.manual_seed(10)
test_size = len(dataset)//5
train_size = len(dataset) - test_size

train_ds, test_ds = random_split(dataset, [train_size, test_size])
len(train_ds), len(test_ds)   

In [ ]:
train_loader=DataLoader(train_ds,batch_size=10,shuffle=True)
test_loader=DataLoader(test_ds,batch_size=10)

# Check Images and Labels

In [ ]:
for images, labels in train_loader:
    break
#print the labels
print('Label:', labels.numpy())
print('Class:', *np.array([class_names[i] for i in labels]))

im=make_grid(images,nrow=5)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
inv_normalize=transforms.Normalize(mean=[-0.485/0.229,-0.456/0.224,-0.406/0.225],
                                    std=[1/0.229,1/0.224,1/0.225])
im=inv_normalize(im)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

# CNN Model

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,6,3,1)
        self.conv2=nn.Conv2d(6,16,3,1)
        self.fc1=nn.Linear(16*54*54,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,20)
        self.fc4=nn.Linear(20,len(class_names))
        
    def forward(self,X):
        #print(X.shape)#torch.Size([10,3,224,224])
        X=F.relu(self.conv1(X))
        X=F.max_pool2d(X,2,2)
        X=F.relu(self.conv2(X))
        X=F.max_pool2d(X,2,2)
        X=X.view(-1,16*54*54)
        X=F.relu(self.fc1(X))
        X=F.relu(self.fc2(X))
        X=F.relu(self.fc3(X))
        X=self.fc4(X)
        X=F.log_softmax(X,dim=1)     
        #print(X.shape)#torch.Size([10,80])
        return X

In [ ]:
mycnn=MyCNN()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(mycnn.parameters(),lr=0.001)

In [ ]:
mycnn

In [ ]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>8}')
    print(f'________\n{sum(params):>8}')
count_parameters(mycnn)

In [ ]:
import time
start_time=time.time()
train_losses=[]
test_losses=[]
train_correct=[]
test_correct=[]
epochs=60

for i in range(epochs):
    trn_corr=0
    tst_corr=0
    for b, (X_train,y_train) in enumerate(train_loader):
        b+=1                                            
        y_pred=mycnn(X_train)
        loss=criterion(y_pred,y_train)
        #true predictions
        predicted=torch.max(y_pred.data,1)[1]
        batch_corr=(predicted==y_train).sum()
        trn_corr+=batch_corr
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%200==0:
            print(f"epoch: {i} loss: {loss.item} batch: {b} accuracy: {trn_corr.item()*100/(10*b):7.3f}%")
    loss=loss.detach().numpy()
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    
    with torch.no_grad():
        for b, (X_test,y_test) in enumerate(test_loader):
            y_val=mycnn(X_test)
            loss=criterion(y_val,y_test)
            
            predicted=torch.max(y_val.data,1)[1]
            btach_corr=(predicted==y_test).sum()
            tst_corr+=btach_corr
            
        loss=loss.detach().numpy()
        test_losses.append(loss)
        test_correct.append(tst_corr)
        
print(f'\nDuration: {time.time() - start_time:.0f} seconds')    

In [ ]:
plt.plot(train_losses,label="train_losses")
plt.plot(test_losses,label="test_losses")
plt.legend()

# Prediction

In [ ]:
device = torch.device("cpu")   #"cuda:0"

In [ ]:
mycnn.eval()
y_true=[]
y_pred=[]
with torch.no_grad():
    for test_data in test_loader:
        test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
        pred = mycnn(test_images).argmax(dim=1)
        for i in range(len(pred)):
            y_true.append(test_labels[i].item())
            y_pred.append(pred[i].item())
print(y_pred[0:5])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred,target_names=class_names,digits=4))

In [ ]:
def simple_inv_normalize(image_tensor):
    # Assuming mean and std were used for normalization during training
    inv_normalize = transforms.Normalize(
        mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
        std=[1 / 0.229, 1 / 0.224, 1 / 0.225]
    )

    # Apply inverse normalization
    inv_img = inv_normalize(image_tensor)
    
    # Ensure the pixel values are in the valid range
    inv_img = torch.clamp(inv_img, 0, 1)

    return inv_img



In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

def visualize_predictions(model, data_loader, class_names):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

    confusion_mat = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 15))
    sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 8})
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
def plot_misclassified(model, data_loader, class_names, num_images=10):
    model.eval()
    misclassified_images = []
    correct_labels = []
    predicted_labels = []

    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)

            misclassified_indices = (predictions != labels).nonzero()
            misclassified_images.extend(images[misclassified_indices])
            correct_labels.extend(labels[misclassified_indices])
            predicted_labels.extend(predictions[misclassified_indices])

            if len(misclassified_images) >= num_images:
                break
    plt.figure(figsize=(15, 15))
    for i in range(len(misclassified_images)):
        plt.subplot(5, 4, i + 1)  # Adjust the number of rows and columns as needed
        inv_img = simple_inv_normalize(misclassified_images[i])
        if inv_img.dim() == 4:
            inv_img = inv_img.squeeze(0)
        plt.imshow(inv_img.permute(1, 2, 0).cpu().numpy())
        plt.title(f'Actual: {class_names[correct_labels[i]]}\nPredicted: {class_names[predicted_labels[i]]}')
        plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize predictions using a confusion matrix
visualize_predictions(mycnn, test_loader, class_names)

# Plot misclassified images
plot_misclassified(mycnn, test_loader, class_names)

